# Obtain geographical data for cohorts

Here we obtain representative cohorts for administrative units and add to the cohorts dataframe.

In [1]:
import pandas as pd
from pyprojroot import here
import geopandas as gpd
import iso3166
import functools

In [2]:
# load cohorts to find sample query 
df_cohorts = pd.read_csv(here() / "build" / "final_cohorts.csv")
df_cohorts.head()

,cohort_id,cohort_size,country,admin1_iso,admin1_name,admin2_name,taxon,year,quarter,cohort_label,sample_query,latitude,longitude,h12_window_size
0,ML-2_Kati_colu_2014_Q3,27,Mali,ML-2,Koulikouro,Kati,coluzzii,2014,3,Mali / Kati / coluzzii / 2014 / Q3,cohort_admin2_quarter == 'ML-2_Kati_colu_2014_...,12.875556,-8.137778,1000
1,ML-2_Kati_gamb_2014_Q3,24,Mali,ML-2,Koulikouro,Kati,gambiae,2014,3,Mali / Kati / gambiae / 2014 / Q3,cohort_admin2_quarter == 'ML-2_Kati_gamb_2014_...,12.888788,-8.149091,2000


## Add ISO3166-1 3-letter country codes

We need 3-letter country codes in order to locate geoboundaries data.

In [3]:
countries = df_cohorts["country"].unique()
countries

array(['Mali'], dtype=object)

In [4]:
len(countries)

1

In [5]:
# some countries known by a different name
countries_by_name = iso3166.countries_by_name
countries_by_name["Democratic Republic of the Congo".upper()] = iso3166.countries_by_alpha3["COD"]
countries_by_name["Cote d'Ivoire".upper()] = iso3166.countries_by_alpha3["CIV"]
countries_by_name["Gambia, The".upper()] = iso3166.countries_by_alpha3["GMB"]
countries_by_name["Tanzania".upper()] = iso3166.countries_by_alpha3["TZA"]

In [6]:
# add columns to the cohorts dataframe
df_cohorts["country_alpha2"] = df_cohorts.apply(
    lambda row: countries_by_name[row.country.upper()].alpha2,
    axis="columns"
)

df_cohorts["country_alpha3"] = df_cohorts.apply(
    lambda row: countries_by_name[row.country.upper()].alpha3,
    axis="columns"
)

In [7]:
df_cohorts[["country", "country_alpha2", "country_alpha3"]].head()

,country,country_alpha2,country_alpha3
0,Mali,ML,MLI
1,Mali,ML,MLI


## Join with geoboundaries data

In [8]:
adm2_geojson_url_template = "https://github.com/wmgeolab/geoBoundaries/raw/v5.0.0/releaseData/gbOpen/{country_alpha3}/ADM2/geoBoundaries-{country_alpha3}-ADM2_simplified.geojson"

In [9]:
@functools.lru_cache(maxsize=None)
def read_adm2(country_alpha3):
    df = gpd.read_file(adm2_geojson_url_template.format(country_alpha3=country_alpha3))
    return df

In [10]:
def country_cohorts_geo(country_alpha3, alias=None):
    """Join cohorts dataframe with admin 2 unit geometry."""
    
    # read the geoboundaries data
    gdf_geo = read_adm2(country_alpha3)
    
    # subset cohorts to country
    df_coh = (
        df_cohorts
        .query(f"country_alpha3 == '{country_alpha3}'")
        .copy()
    )
    
    # deal with admin2 name aliases
    if alias is not None:
        df_coh["admin2_name"].replace(alias, inplace=True)
            
    # join dataframes
    gdf_ret = gpd.GeoDataFrame(
        df_coh.merge(
            gdf_geo, 
            left_on=["country_alpha3", "admin2_name"],
            right_on=["shapeGroup", "shapeName"],
            how="left",
        )
    )
    
    # confirm no missing data
    loc_missing = gdf_ret["shapeID"].isna()
    if loc_missing.any():
        print(
            "ERROR",
            country_alpha3,
            gdf_ret.iloc[loc_missing.values]["admin2_name"].unique()
        )
        
    # add representative point
    point = gdf_ret.representative_point()
    gdf_ret["representative_lon"] = point.x
    gdf_ret["representative_lat"] = point.y
    
    
    return gdf_ret

In [11]:
country_codes = df_cohorts["country_alpha3"].unique()
country_codes

array(['MLI'], dtype=object)

In [12]:
# define aliases because cohorts were originally defined
# using geoboundaries v3, but here we are using v5
adm2_alias = {
    
    # CMR - Cameroon
    "Lom-Et-Djérem": "Lom-Et-Djerem",
    
    # GHA - Ghana
    "Twifu Heman/Lower Denkyira": "Twifo Hemang Lower Denkyira",
    # N.B., this is a hack, New Juaben has been split into two
    # districts. But in Ag3.0 we only have data from Koforidua
    # which is the capital of New Juaben South.
    "New Juaben": "New Juaben South Municipal",  
    # N.B., this is a hack, Shama Ahanta East has been split
    # into Shama and Sekondi Takoradi. In Ag3.0 we only have
    # data from Takoradi.
    "Shama Ahanta East": "Sekondi Takoradi Metropolis",
    
    # GMB - The Gambia
    # N.B., hack, in Ag3.0 location is Wali Kunda, checked on
    # map that it's in Lower Fuladu West.
    "Fulladu West": "Lower Fuladu West",
    "Central Baddibu": "Central Badibu",
    
    # GNB - Guinea-Bissau
    "Setor De Safim": "Setor de Safim",
    
    # KEN - Kenya
    # N.B., hack, in Ag.30 checked coordinates, location is
    # in Kilifi North
    "Kilifi": "Kilifi North",
    
}

gdf_cohorts = pd.concat(
    [country_cohorts_geo(x, alias=adm2_alias)
     for x in country_codes]
)
gdf_cohorts.head()

,cohort_id,cohort_size,country,admin1_iso,admin1_name,admin2_name,taxon,year,quarter,cohort_label,...,country_alpha2,country_alpha3,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry,representative_lon,representative_lat
0,ML-2_Kati_colu_2014_Q3,27,Mali,ML-2,Koulikouro,Kati,coluzzii,2014,3,Mali / Kati / coluzzii / 2014 / Q3,...,ML,MLI,Kati,,8926073B54119460885487,MLI,ADM2,"POLYGON ((-9.04476 12.42636, -9.02295 12.41940...",-8.426097,12.598369
1,ML-2_Kati_gamb_2014_Q3,24,Mali,ML-2,Koulikouro,Kati,gambiae,2014,3,Mali / Kati / gambiae / 2014 / Q3,...,ML,MLI,Kati,,8926073B54119460885487,MLI,ADM2,"POLYGON ((-9.04476 12.42636, -9.02295 12.41940...",-8.426097,12.598369


In [13]:
gdf_cohorts.to_file(here() / "build" / "final_cohorts.geojson")

## Check

Check the file reads back in.

In [14]:
gdf_check = gpd.read_file(here() / "build" / "final_cohorts.geojson").head()
gdf_check.head()

,cohort_id,cohort_size,country,admin1_iso,admin1_name,admin2_name,taxon,year,quarter,cohort_label,...,country_alpha2,country_alpha3,shapeName,shapeISO,shapeID,shapeGroup,shapeType,representative_lon,representative_lat,geometry
0,ML-2_Kati_colu_2014_Q3,27,Mali,ML-2,Koulikouro,Kati,coluzzii,2014,3,Mali / Kati / coluzzii / 2014 / Q3,...,ML,MLI,Kati,,8926073B54119460885487,MLI,ADM2,-8.426097,12.598369,"POLYGON ((-9.04476 12.42636, -9.02295 12.41940..."
1,ML-2_Kati_gamb_2014_Q3,24,Mali,ML-2,Koulikouro,Kati,gambiae,2014,3,Mali / Kati / gambiae / 2014 / Q3,...,ML,MLI,Kati,,8926073B54119460885487,MLI,ADM2,-8.426097,12.598369,"POLYGON ((-9.04476 12.42636, -9.02295 12.41940..."


## Debug

Code cells below were used to debug the aliases required to join admin2 units.

In [ ]:
# import malariagen_data
# ag3 = malariagen_data.Ag3()

In [ ]:
# (
#     ag3
#     .sample_metadata()
#     .query("admin2_name == 'Kilifi'")
#     .groupby(["location", "latitude", "longitude"])
#     .size()
# )

In [ ]:
# read_adm2("KEN").explore()

In [ ]:
# with pd.option_context("display.max_rows", None):
#     display(read_adm2("KEN"))